In [48]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
tokens = text.encode("utf-8")
tokens = list(map(int,tokens))

tokens

[239,
 188,
 181,
 239,
 189,
 142,
 239,
 189,
 137,
 239,
 189,
 131,
 239,
 189,
 143,
 239,
 189,
 132,
 239,
 189,
 133,
 33,
 32,
 240,
 159,
 133,
 164,
 240,
 159,
 133,
 157,
 240,
 159,
 133,
 152,
 240,
 159,
 133,
 146,
 240,
 159,
 133,
 158,
 240,
 159,
 133,
 147,
 240,
 159,
 133,
 148,
 226,
 128,
 189,
 32,
 240,
 159,
 135,
 186,
 226,
 128,
 140,
 240,
 159,
 135,
 179,
 226,
 128,
 140,
 240,
 159,
 135,
 174,
 226,
 128,
 140,
 240,
 159,
 135,
 168,
 226,
 128,
 140,
 240,
 159,
 135,
 180,
 226,
 128,
 140,
 240,
 159,
 135,
 169,
 226,
 128,
 140,
 240,
 159,
 135,
 170,
 33,
 32,
 240,
 159,
 152,
 132,
 32,
 84,
 104,
 101,
 32,
 118,
 101,
 114,
 121,
 32,
 110,
 97,
 109,
 101,
 32,
 115,
 116,
 114,
 105,
 107,
 101,
 115,
 32,
 102,
 101,
 97,
 114,
 32,
 97,
 110,
 100,
 32,
 97,
 119,
 101,
 32,
 105,
 110,
 116,
 111,
 32,
 116,
 104,
 101,
 32,
 104,
 101,
 97,
 114,
 116,
 115,
 32,
 111,
 102,
 32,
 112,
 114,
 111,
 103,
 114,
 97,
 109,
 109,
 101

In [49]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): ## zip() makes tuples of consequent symbols and the loop goes trough these tuples
        counts[pair] = counts.get(pair,0) + 1 ## get() method gets the `pair` element of counts, else if there are no such elements it just returns 0 (here 0 is the default value of return)
                                              ## The Key of 'pair' is the number apearances of the 'pair' which we increasing by 1
    return counts
stats = get_stats(tokens)

In [50]:
top_pair = max(stats, key = stats.get)

In [51]:
def merge(ids, pair, idx):
    ## ids is ints list. Here we will replace all consecsecuentive occurences of pair with the new token idx
    newids = []
    i = 0
    while i < len(ids) - 1:
        if ids[i] == pair[0] and ids[i + 1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i]) ## If no matching pair we just copy initial token to newids
            i += 1
    return newids
    
tokens2 = merge(tokens, top_pair, 256)


In [52]:
vocab_size = 276
num_merges = vocab_size - 256
ids = list(tokens) ## Here we remove link between initial tokens array and ids (see mutable objects in python)

merges = {} # (int, int) -> int

for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key = stats.get)
    idx = 256 + i
    print(f"Merging {pair} into a new token {idx}")
    ids = merge(ids, pair, idx)
    merges[pair] = idx 

## In this section we trained our tokenizer and maked the merging vocabulary which we will use in future

Merging (101, 32) into a new token 256
Merging (240, 159) into a new token 257
Merging (226, 128) into a new token 258
Merging (105, 110) into a new token 259
Merging (115, 32) into a new token 260
Merging (97, 110) into a new token 261
Merging (116, 104) into a new token 262
Merging (257, 133) into a new token 263
Merging (257, 135) into a new token 264
Merging (97, 114) into a new token 265
Merging (239, 189) into a new token 266
Merging (258, 140) into a new token 267
Merging (267, 264) into a new token 268
Merging (101, 114) into a new token 269
Merging (111, 114) into a new token 270
Merging (116, 32) into a new token 271
Merging (259, 103) into a new token 272
Merging (115, 116) into a new token 273
Merging (261, 100) into a new token 274
Merging (32, 262) into a new token 275


In [61]:
''' MY DECODING VARIANT 
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0,p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]
def decode(ids):
    raw = b""
    for i in ids:
        raw += vocab[i]
    print(raw)
    text = raw.decode("utf-8", errors = "replace")
    return text
'''


vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1] ## vocab[p0] and vocab[p1] have "bytes" type so their sum will concetenate them. (For example a = b'\x02' and c= b'\x01' then a+b = b'\x02\x01')
def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids) ## Getting raw byte stream
    text = tokens.decode("utf-8", errors = "replace") ##Here can be some problems with utf-8 standard so there can be some issues when decoding and program will throw error. In this purposses we set errors="replace"
    return text

In [56]:
'''
def encode(text):
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key = lambda p: merges.get(p,float("inf"))) ###??????????
        if pair not in merges:
            break ## nothing else can be merged
        idx = merges[pair] ## Getting the index which will replace the merged sequence ids
        tokens = merge(tokens, pair, idx)
    return tokens
'''

def encode(text):
    tokens = list(text.encode("utf-8"))
    stats = get_stats(tokens)

    return tokens


'Vova'